Цель модуля - парсить следующую инфу, которой нет в обычной выгрузке: 
* цену контракта после снижения
* победителя
* 


In [2]:
import requests
from bs4 import BeautifulSoup as BS
import bs4
import time
from fake_useragent import UserAgent
import re 
#from decimal import Decimal
import numpy as np
import pandas as pd
import shutil
import os

In [9]:
link_main_pattern = 'http://zakupki.gov.ru'
# link_pattern_fz44_order_info = 'http://zakupki.gov.ru/epz/order/notice/ea44/view/common-info.html?regNumber='
link_pattern_fz44_result_info = 'http://zakupki.gov.ru/epz/order/notice/ea44/view/supplier-results.html?regNumber='
# link_pattern_fz44_documents = 'http://zakupki.gov.ru/epz/order/notice/ea44/view/documents.html?regNumber='
# link_pattern_fz44_contract_info = 'http://zakupki.gov.ru/epz/contract/extendedsearch/results.html?searchString=&orderNumber='
link_pattern_fz44_contruct = 'http://zakupki.gov.ru/epz/contract/contractCard/document-info.html?reestrNumber='
link_pattern_fz223 = 'http://zakupki.gov.ru/223/purchase/public/purchase/info/common-info.html?regNumber='

### Перечень локальных функций:

In [10]:
def get_page(link):
    page = requests.get(link, headers={'User-Agent': UserAgent().chrome})
    page.encoding = 'utf8'
    page = page.text
    soup = BS(page)
    while soup.title.text == ' Страница не найдена ':
        time.sleep(5)
        get_page(link)
    return soup

In [11]:
def clear_text(content):
    content = re.sub(r'[\n]+', '', content) # Очистили от переносов
    content = re.sub(r'[\s]{2,}', '', content) # Очистили от лишних пробелов
    content = re.sub(r'\xa0', '', content) # Соединили цену
    content = re.sub(r',', '.', content) # изменили запись для возможности преобрадования str в decimal
    content = re.sub(r' Российский рубль', '', content) # убираем валюту
    return content

In [232]:
def get_win_info_44(order_num):
    link = link_pattern_fz44_result_info + order_num
    soup = get_page(link)
    winner = soup.find_all('td')[-3].text # опытным путём определил, где находится наименование победителя
    winner = clear_text(winner)
    
    winner_price = soup.find_all('td')[-2].text
    winner_price = clear_text(winner_price)
    if winner_price = '':
        winner_price = float(winner_price)

        contruct_link_div = soup.find_all('a', href=True)[-1]
        link_to_contruct_info = link_main_pattern + contruct_link_div.get('href') # парсим ссылку на страницу
        num_of_contruct = contruct_link_div.text.replace('№','') #  парсим номер контракта и освобождаем его от значка №


        df = pd.DataFrame({
        'Реестровый номер закупки': [order_num],
        'победитель': [winner],
        'цена контракта после снижения': [winner_price],
        'номер контракта': [num_of_contruct]
        })
    
        return df
    

In [17]:
def get_empty_dataframe():
    df = pd.DataFrame({
    'Реестровый номер закупки': [],
    'победитель': [],
    'цена контракта после снижения': [],
    'номер контракта': []
    })
    
    return df

In [129]:
def get_contruct(contruct_num):
    link = link_pattern_fz44_contruct + contruct_num
    soup = get_page(link)

    contruct_div = soup.find_all('a', class_='')[2] 
    link_to_contruct = contruct_div.get('href')
    file_name = contruct_div.get('title')

    file_extension = re.findall(r'\.[a-z]+', file_name)[0]
    directory = 'contructs/' 
    filename =  directory + contruct_num + file_extension
    file = requests.get(link_to_contruct, stream=True, headers={'User-Agent': UserAgent().chrome})
    
    if file.status_code == 200:
        with open(filename, 'wb') as f:
            file.raw.decode_content = True
            shutil.copyfileobj(file.raw, f)

### * * * Сборочный стол * * *

In [207]:
order_num = '0149200002318006192'

### * * * THE END * * *

### Проверка

In [151]:
test_data = [ 
'0349100004618000225',
'0349100004618000224',
'0349100004618000228',
'0149200002318006192',
'0349100021118000118',
'0149200002318005842',
'0349200022018000059',
'0149200002318005863',
'0149200002318005050',
'0149200002318005280',
'0149200002318005233',
'0149200002318005238',
'0149200002318005091',
'0149200002318005081',
'0149200002318004994',
'0149200002318005000',
'0349200017118000031',
'0149200002318004967',
'0149200002318004616',
'0149200002318004712']

test_data = [ 
'0349100004618000225',
'0349100004618000224',
'0349100004618000228',
'0149200002318006192',
'0349100021118000118',
'0149200002318005842',
'0349200022018000059',
'0149200002318005863']

In [233]:
df = get_empty_dataframe()
for order in test_data:
    tmp_df = get_win_info_44(order)
    df= pd.concat([df, tmp_df])
    time.sleep(2)

In [234]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18 entries, 0 to 0
Data columns (total 4 columns):
Реестровый номер закупки         18 non-null object
победитель                       18 non-null object
цена контракта после снижения    18 non-null float64
номер контракта                  18 non-null object
dtypes: float64(1), object(3)
memory usage: 720.0+ bytes


In [236]:
df

,Реестровый номер закупки,победитель,цена контракта после снижения,номер контракта
0,0349100004618000225,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""СОРЭ...",512524.50,1511200012819000036
0,0349100004618000224,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ГРАН...",875440.80,1511200012819000042
0,0349100004618000228,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ГРАН...",49183.20,1511200012819000029
0,0349100021118000118,"Общество с ограниченной ответственностью ""Сорэ...",516210.92,1519005315919000016
0,0149200002318005842,ООО 'СорэксМед',572882.00,2519214003019000009
0,0349200022018000059,Общества с ограниченной ответственностью 'Сорэ...,308880.00,2519006077319000008
0,0149200002318005863,"ЗАО ""Мурманский универсальный завод""",689509.80,2510890002019000002
0,0149200002318005050,"ООО ""ОРКО-ЦЕНТР""",147060.00,2519150067418000350
0,0149200002318005280,ООО 'СорэксМед',1095789.76,2519040415018000092
0,0149200002318005233,"ООО ""ОРКО-ЦЕНТР""",39825.00,2519011131518000067


# ПОМОЙКА

In [ ]:
tag = soup.td
tag

In [ ]:
tag.attrs

In [ ]:
tag.contents

In [ ]:
for child in tag.descendants:
    print(child)

In [ ]:
len(list(soup.children))

In [ ]:
len(list(soup.descendants))

In [ ]:
soup.find_all('div', class_='contentTabBoxBlock') # нашёл бокс с интересующей инфой

In [ ]:
b = soup.find_all('td')
for i in range(len(b)):
    print(f'**************{i}****************')
    print(b[i])

In [ ]:
b[16].text # winner

In [ ]:
soup.find_all('td')[16].text

# СПРАВОЧНИК

In [ ]:
# soup.find_all('a')  - все ссылки

### Интересная штучка из  http://qaru.site/questions/15235681/beautifulsoup4-documentation-example-doesnt-work/24168590  для поиска определённых тагов

In [ ]:
def has_class_but_no_id(tag):
    return tag.has_attr('class') and not tag.has_attr('id')
a.find_all(has_class_but_no_id)

In [ ]:
# образец кода для парсинга слов в вики (первого порядка)
def get_links(topic):
    link = 'https://ru.m.wikipedia.org/wiki/' + topic.capitalize()
    spam = get(link).text
    soup = BS(spam, 'html.parser')
    link_example = """ <a href="LINK">CONTENT</a> """
    agg = soup.find_all("a")
    links = []
    for n in agg:
        lnk = n.get("href")
        if re.fullmatch(r'\/wiki\/(?:)[^\',\n]*|\/w\/(?:)[^\',\n]*', lnk) is not None:
            links.append('https://ru.m.wikipedia.org' + lnk)
        else:
            continue
    links.append(link)

    return links

In [45]:
# цикл, возвращающий ссылки:
for i in soup.find_all('a', href=True):
    print(i['href'])

/epz/main/public/home.html
/epz/order/notice/rss?regNumber=0349200022018000059
/epz/order/notice/printForm/view.html?printFormId=82362759
/epz/order/notice/zk44/view/protocol/protocol-main-info.html?regNumber=0349200022018000059&protocolId=21877585
/epz/contract/signview/list.html?printFormId=167266275
/epz/contract/printForm/view.html?printFormId=167266275
/epz/contract/contractCard/common-info.html?reestrNumber=2519006077319000008
